In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from dataset import Model10NetDataset
from model import TNet, PointNet

In [3]:
torch.cuda.is_available()

True

In [4]:
dataset = Model10NetDataset(path="./ModelNet10")

In [5]:
point_cloud = dataset[0][0]
point_cloud.shape

[[0, 1, 2], [3, 4, 5], [6, 7, 8], [7, 6, 9], [10, 11, 12], [11, 10, 13], [13, 10, 14], [13, 14, 7], [7, 14, 8], [8, 14, 15], [15, 6, 8], [6, 15, 16], [17, 18, 19], [18, 17, 20], [18, 20, 21], [21, 20, 9], [21, 9, 6], [21, 6, 16], [9, 13, 7], [13, 9, 20], [17, 12, 11], [12, 17, 19], [19, 10, 12], [10, 19, 18], [18, 14, 10], [14, 18, 21], [15, 21, 16], [21, 15, 14], [13, 17, 11], [17, 13, 20], [24, 25, 26], [27, 24, 26], [24, 28, 25], [29, 26, 25], [27, 26, 30], [31, 24, 27], [31, 28, 24], [28, 32, 25], [29, 30, 26], [29, 25, 32], [27, 30, 33], [31, 27, 33], [31, 34, 28], [28, 34, 32], [29, 35, 30], [34, 29, 32], [33, 30, 35], [31, 33, 36], [31, 37, 34], [38, 29, 39], [29, 38, 35], [34, 37, 29], [33, 35, 36], [31, 36, 40], [31, 41, 37], [40, 38, 39], [36, 35, 38], [42, 29, 43], [29, 42, 39], [37, 41, 29], [36, 38, 40], [31, 40, 44], [31, 45, 41], [40, 39, 44], [46, 42, 43], [47, 39, 42], [29, 48, 43], [41, 45, 29], [31, 44, 47], [31, 49, 45], [44, 39, 47], [46, 43, 48], [47, 42, 46], [50

torch.Size([1601, 3])

In [6]:
dataset[1][0].shape

[[0, 1, 2], [1, 0, 3], [4, 5, 6], [5, 4, 7], [5, 7, 8], [8, 7, 9], [8, 10, 11], [10, 8, 9], [12, 13, 14], [13, 12, 15], [15, 12, 16], [16, 12, 17], [16, 17, 18], [18, 17, 19], [13, 20, 14], [20, 13, 21], [10, 22, 11], [22, 10, 23], [22, 23, 20], [22, 20, 21], [24, 25, 26], [25, 24, 27], [28, 29, 30], [29, 28, 31], [31, 28, 32], [32, 28, 33], [32, 34, 35], [34, 32, 33], [34, 36, 35], [36, 34, 37], [36, 37, 38], [36, 38, 39], [39, 38, 40], [39, 40, 41], [29, 42, 30], [42, 29, 43], [44, 45, 46], [45, 44, 42], [45, 42, 43], [47, 46, 45], [48, 49, 50], [49, 48, 51], [52, 53, 54], [53, 52, 55], [56, 57, 58], [57, 56, 59], [60, 61, 62], [61, 60, 63], [64, 65, 66], [65, 64, 67], [68, 69, 70], [69, 68, 71], [72, 73, 74], [73, 72, 75], [76, 77, 78], [77, 76, 79], [80, 81, 82], [81, 80, 83], [84, 85, 86], [85, 84, 87], [88, 89, 90], [89, 88, 91], [92, 93, 94], [93, 92, 95], [96, 97, 98], [97, 96, 99], [100, 101, 102], [101, 100, 103], [104, 105, 106], [105, 104, 107], [108, 109, 110], [109, 108, 

torch.Size([124, 3])

In [21]:
point_cloud = point_cloud.to("cuda")
point_cloud.shape

torch.Size([1601, 3])

In [22]:
def stack(x):
    # Add a dimension to make it (1, N, C)
    x_unsqueezed = x.unsqueeze(0)

    # Concatenate along the new dimension to make it (2, N, C)
    x_stacked = torch.cat([x_unsqueezed, x_unsqueezed], dim=0)

    return x_stacked

In [23]:
point_cloud = stack(point_cloud)
point_cloud = point_cloud.permute((0, 2, 1))
point_cloud.shape

torch.Size([2, 3, 1601])

In [24]:
tnet = TNet(in_features=3)
tnet = tnet.to("cuda")
tnet

TNet(
  (mlp): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=9, bias=True)
  )
  (batchnorm): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=

In [25]:
out = tnet(point_cloud)
out.shape

torch.Size([2, 3, 3])

In [ ]:
out

tensor([[[0.0000, 1.2749, 0.0000,  ..., 0.0000, 0.2662, 1.9018],
         [0.0000, 0.9713, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5344, 0.0000, 0.6598,  ..., 0.0000, 0.3981, 1.0012],
         ...,
         [0.9243, 0.0000, 0.0000,  ..., 0.2911, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5123,  ..., 0.7256, 0.0000, 0.0000],
         [0.6774, 0.4034, 0.0000,  ..., 0.0000, 0.8900, 0.4463]],

        [[0.0000, 1.2749, 0.0000,  ..., 0.0000, 0.2662, 1.9018],
         [0.0000, 0.9713, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5344, 0.0000, 0.6598,  ..., 0.0000, 0.3981, 1.0012],
         ...,
         [0.9243, 0.0000, 0.0000,  ..., 0.2911, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5123,  ..., 0.7256, 0.0000, 0.0000],
         [0.6774, 0.4034, 0.0000,  ..., 0.0000, 0.8900, 0.4463]]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [11]:
pointnet = PointNet()
pointnet = pointnet.to("cuda")
pointnet

PointNet(
  (input_transform): TNet(
    (mlp): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
    (fc): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=256, out_features=9, bias=Tru

In [13]:
x = pointnet(point_cloud)
x[0].shape, x[1].shape, x[2].shape

(torch.Size([2, 10]), torch.Size([2, 3, 3]), torch.Size([2, 64, 64]))